 # Start

### Importing the Libraries

In [1]:
import sys
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
import pandas as pd

from ESN_Cell import *
from ddeint import ddeint

import os
os.getcwd()

'/home/tah/Documents/Computation_EOC/esn-neuroevolution'

### Resetting the Graph

In [2]:
tf.reset_default_graph()

### Initializing the DataFrame to store Results

In [3]:
df = pd.DataFrame(columns=['n','Spectral_Radius','MSE','alpha'])

## Simulating the Mackey-Glass Equation
#### for 6 different 'n' values

In [4]:
model = lambda X,t,beta,n,tau,gamma : beta*((X(t-tau))/(1 + X(t-tau)**n)) - gamma*X(t)

X_0 = lambda t:0.5 # history before t=0

X_ts = {}

bifurcation_para = [4.5, 6, 7.5, 8.5, 9.5, 9.6, 9.65, 9.7, 9.8, 9.9, 10, 11, 15, 20]

for i, n in enumerate(bifurcation_para):
    t = linspace(0,100,10000)
    X_t = ddeint(model, X_0, t, fargs=(2, n, 2, 1)) #beta=2, n=n, tau=2, gamma=1
    X_ts[n] = X_t    

#### Plotting the input

In [ ]:
plt.figure(figsize=(120,120))
plt.rcParams.update({'font.size': 25})

for i, n in enumerate(bifurcation_para):
    plt.subplot(7,2,i+1)
    plt.plot(t, X_ts[n],label= "n = {}".format(n), color='black', lw=5.0)
    plt.xlabel("Time_t")
    plt.ylabel("X_t")
    plt.legend(loc="lower right", prop={'size': 100})

plt.show()


In [5]:
batch_size = 1
length_ts = len(X_ts[4.5])

X_ts_ = {}

for _,n in enumerate(X_ts):
    X_ts_[n] = np.reshape(X_ts[n], [batch_size, length_ts, 1]) #reshaped to: [batch_size, time, elements]


## Defining ESN Architecture Parameters

In [6]:
res_units = 100
in_units = 1

ESN_arch = [in_units, res_units]

## Defining ESN Variables/Parameters

In [23]:
leak_rate = 0.2
#Lower value of leak_rate (alpha) denotes lower influence of current state x(n) and higher influence of
#previous state x(n-1) in determination of the next state: X(n) = (1-alpha)*x(n-1) + alpha*x(n)

activation = tf.tanh
weights_variance = 0.1
sparseness = True
sparsity = 0.1 # 10% connectivity in the reservoir

## Instantiating the ESN Cell

In [24]:
esn = ESN(ESN_arch, activation, leak_rate, weights_variance, sparsity, sparseness)

## Computing ESN States

In [25]:
spectral_rad = esn.__dict__['spectral_radius'] #value of the spectral radius of the 'esn'
w_res = esn.__dict__['weights_res']
w_in = esn.__dict__['weights_in']

#Placeholders for feeding during session run
inputs = tf.placeholder(tf.float32, [batch_size, length_ts, in_units])
init_state = tf.placeholder(tf.float32, [1, res_units])


#Using the tensorflow API to compute the output of the ESN
output_state, last_state = tf.nn.dynamic_rnn(esn, inputs, initial_state=init_state)
print(output_state.shape, last_state.shape)

outputs = tf.reshape(output_state, [length_ts, res_units])

(1, 10000, 100) (1, 100)


## Defining Training Parameters

In [26]:
#Training on the first 5000 time steps for future prediction
#And, using the remaining 5000 time steps for testing on predictions
train_size = 5000

#length of initial transient
init_transient = 1000

# training data
train_states = tf.cast(outputs[init_transient:train_size+init_transient], dtype="float64")
train_target = tf.cast(tf.squeeze(inputs[:, init_transient+1:train_size+init_transient+1, :], axis=0), \
                       dtype="float64")
print(train_states.shape, train_target.shape)

#test data
test_states = tf.cast(outputs[train_size+init_transient:], dtype="float64")
test_target = tf.cast(tf.squeeze(inputs[:, train_size+init_transient+1:, :], axis=0), dtype="float64")
print(test_states.shape, test_target.shape)

(5000, 100) (5000, 1)
(4000, 100) (3999, 1)


## Ridge Regression to compute Output

In [27]:
beta = 0.1 #'Beta' value for regularization in ridge regression

X_XT = tf.matmul(tf.transpose(train_states),train_states) #X_XT is 100x100
B_I = tf.cast(tf.scalar_mul(beta, tf.eye(int(outputs.shape[1]))), dtype="float64") #B_I is 100x100

weights_out = tf.matmul(tf.matmul(tf.transpose(train_target), train_states), tf.linalg.inv(X_XT + B_I))

weights_out = tf.transpose(weights_out)
print(weights_out.shape)

#Computing Predictions
test_out = tf.matmul(test_states, weights_out)[:-1]
print(test_out.shape)

norm_test_target = tf.math.l2_normalize(test_target)
norm_test_out = tf.math.l2_normalize(test_out)

#Computing Mean Squared Loss
MSE = tf.losses.mean_squared_error(norm_test_target, norm_test_out)

(100, 1)
(3999, 1)


## Training the ESN for each MGE series

In [28]:
# bifurcation_para = [4.5, 6, 7.5, 8.5, 9.5, 9.6, 9.65, 9.7, 9.8, 9.9, 10, 11, 15, 20]

check_n = []
groupsize = 10

with tf.Session() as sess:
        
        sess.run(tf.global_variables_initializer())
        
        init_esn_state = np.zeros([1, res_units], dtype="float32")              
        
        outputs_dict = {}
        spec_rad_dict = {}
        mse_dict = {}
        
        plt.rcParams.update({'font.size': 12})
        
        for i, n in enumerate(X_ts_):
            
            spec_rad, w_r, w_i, outs, mse, ts_target, ts_out = sess.run([spectral_rad, w_res, w_in,\
                                                               outputs,MSE,test_target,test_out],\
                                                               feed_dict={inputs:X_ts_[n],\
                                                                          init_state:init_esn_state})
            
            df.loc[i+1] = [n, spec_rad, mse, leak_rate]
                
            if n in check_n:
                print("Mean Squared Error for n = {} : {}".format(n, mse))
        
                plt.figure(figsize=(20,5))
                plt.subplot(121)
                plt.plot(t[train_size+init_transient+1:], ts_target)
                plt.plot(t[train_size+init_transient+1:], ts_out)
                plt.title("for n = {}".format(n), loc='center')
                plt.xlabel("Time_t")
                plt.ylabel("Output of ESN & Original MG-Data".format(res_units))
                plt.legend(("Original MG","ESN Output"), loc = "upper right")
                plt.subplot(122)
                plt.plot(t, X_ts[n],label= "n = {}".format(n), color='black', lw=1.0)
                plt.title("for n = {}".format(n), loc='center')
                plt.xlabel("Time_t")
                plt.ylabel("X_t")

                start = 0
                plt.figure(figsize=(30,20))
                for i in range(groupsize):
                    plt.subplot(4,3,i+1)
                    plt.plot(t, outs[:,start:start+groupsize-1])
                    plt.xlabel("Time_t")
                    plt.ylabel("Output of {} Reservoir Neurons".format(int(res_units/groupsize)))
                    plt.ylim(-0.5, 0.5)
                    start = start+groupsize
                    
                plt.show()
            
df

,n,Spectral_Radius,MSE,alpha
1,4.50,0.999999,1.344503e-12,0.2
2,6.00,0.999999,1.985546e-08,0.2
3,7.50,0.999999,2.976341e-08,0.2
4,8.50,0.999999,3.452999e-08,0.2
5,9.50,0.999999,3.644631e-08,0.2
6,9.60,0.999999,3.652200e-08,0.2
7,9.65,0.999999,3.798030e-08,0.2
8,9.70,0.999999,3.670293e-08,0.2
9,9.80,0.999999,4.041429e-08,0.2
10,9.90,0.999999,4.327103e-08,0.2


# END

In [13]:
df_2_1 = df
print(df_2_1)

w_reservoir_1 = w_r
w_input_1 = w_i

        n  Spectral_Radius           MSE  alpha
1    4.50         0.999999  1.344503e-12    0.2
2    6.00         0.999999  1.985546e-08    0.2
3    7.50         0.999999  2.976341e-08    0.2
4    8.50         0.999999  3.452999e-08    0.2
5    9.50         0.999999  3.644631e-08    0.2
6    9.60         0.999999  3.652200e-08    0.2
7    9.65         0.999999  3.798030e-08    0.2
8    9.70         0.999999  3.670293e-08    0.2
9    9.80         0.999999  4.041429e-08    0.2
10   9.90         0.999999  4.327103e-08    0.2
11  10.00         0.999999  3.616908e-08    0.2
12  11.00         0.999999  4.989440e-08    0.2
13  15.00         0.999999  5.331168e-08    0.2
14  20.00         0.999999  5.750641e-08    0.2


In [29]:
df_5_1 = df
print(df_5_1)

w_reservoir_2 = w_r
w_input_2 = w_i

print(w_reservoir_1 == w_reservoir_2, '\n\n', w_input_1 == w_input_2)

        n  Spectral_Radius           MSE  alpha
1    4.50         0.999999  1.344503e-12    0.2
2    6.00         0.999999  1.985546e-08    0.2
3    7.50         0.999999  2.976341e-08    0.2
4    8.50         0.999999  3.452999e-08    0.2
5    9.50         0.999999  3.644631e-08    0.2
6    9.60         0.999999  3.652200e-08    0.2
7    9.65         0.999999  3.798030e-08    0.2
8    9.70         0.999999  3.670293e-08    0.2
9    9.80         0.999999  4.041429e-08    0.2
10   9.90         0.999999  4.327103e-08    0.2
11  10.00         0.999999  3.616908e-08    0.2
12  11.00         0.999999  4.989440e-08    0.2
13  15.00         0.999999  5.331168e-08    0.2
14  20.00         0.999999  5.750641e-08    0.2
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  T

In [21]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

#print(w_reservoir_1, '\n\n', w_reservoir_2)
#print(w_input_1, '\n\n', w_input_2)